In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("lr_example").getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
# data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)
data = sqlContext.sql("SELECT * FROM Ecommerce_Customers_csv")

In [5]:
data.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)

In [6]:
for item in data.head(2)[1]:
  print(item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

Out[11]: ['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [9]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership'], 
                           outputCol='features')

In [10]:
output = assembler.transform(data)

In [11]:
output.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)
-- features: vector (nullable = true)

In [12]:
final_data = output.select('features', 'Yearly Amount Spent')

In [13]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [14]:
train_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 351|
 mean| 500.28266165574064|
 stddev| 82.32611095381759|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [15]:
test_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 149|
 mean| 497.0322475713441|
 stddev| 71.9358461826447|
 min| 327.37795258965207|
 max| 744.2218671047146|
+-------+-------------------+

In [16]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [17]:
lrModel = lr.fit(train_data)

In [18]:
test_results = lrModel.evaluate(test_data)

In [19]:
test_results.residuals.show()

+--------------------+
 residuals|
+--------------------+
 10.329049752589526|
 6.758775632585866|
 4.239604154587937|
 -6.044915672358741|
 -0.6780351878581996|
 -3.7346918706111865|
 -5.418398217086406|
 3.7865862798788044|
 -17.36028819366743|
 -1.8616570643727073|
 17.095293400915182|
 6.721756139997922|
 -7.127891428710086|
 -2.4854347854295042|
 -9.372772967565595|
 -17.245347799003923|
 -2.138163118391901|
 -13.994567751254635|
 -17.710231627013172|
-0.11653444683571479|
+--------------------+
only showing top 20 rows

In [20]:
test_results.rootMeanSquaredError

Out[24]: 9.704898334027554

In [21]:
test_results.r2

Out[25]: 0.9816761882989757

In [22]:
final_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 500|
 mean| 499.3140382585909|
 stddev| 79.3147815497068|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [23]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
 features|
+--------------------+
[29.5324289670579...|
[30.9716756438877...|
[31.0472221394875...|
[31.0613251567161...|
[31.2606468698795...|
[31.4252268808548...|
[31.5147378578019...|
[31.5316044825729...|
[31.5702008293202...|
[31.5761319713222...|
[31.6098395733896...|
[31.6548096756927...|
[31.7207699002873...|
[31.8186165667690...|
[31.8854062999117...|
[31.9048571310136...|
[31.9120759292006...|
[31.9365486184489...|
[31.9563005605233...|
[32.0047530203648...|
+--------------------+
only showing top 20 rows

In [24]:
predictions = lrModel.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[29.5324289670579...|398.31130132003796|
[30.9716756438877...|487.87983412430685|
[31.0472221394875...|388.25779503443346|
[31.0613251567161...|493.60037373026034|
[31.2606468698795...| 422.0046664448096|
[31.4252268808548...| 534.5014105253731|
[31.5147378578019...| 495.2308862135478|
[31.5316044825729...|432.72901944948376|
[31.5702008293202...| 563.3057803350723|
[31.5761319713222...| 543.088241053701|
[31.6098395733896...| 427.450256250193|
[31.6548096756927...| 468.5416675875506|
[31.7207699002873...| 545.902824906733|
[31.8186165667690...|448.90410815556515|
[31.8854062999117...| 399.4760459400411|
[31.9048571310136...|491.19520522182006|
[31.9120759292006...| 389.6728794240996|
[31.9365486184489...|441.19395264658283|
[31.9563005605233...| 564.836163374212|
[32.0047530203648...|463.86251556746515|
+--------------------+------------------+
only showing top 20 rows